In [ ]:
import numpy as np
import sys
sys.path.append("../")
from algorithm.koopmanlib.dictionary import PsiNN

In [ ]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=25)

In [ ]:
data_path = r'D:\Residual-Dynamic-Mode-Decomposition-main\Examples_gallery_1\ResDMD_datasets' 
import scipy.io
# temp = scipy.io.loadmat(data_path+'pendulum_data.mat')
temp = scipy.io.loadmat(data_path + '\\data_pendulum.mat')
# Y = np.concatenate((temp['pbn_lfp'],temp['lgn_lfp']),axis=1)[1
X = temp['DATA_X']
Y = temp['DATA_Y']

print(X.shape)
print(type(X))


In [ ]:
del temp

In [ ]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [ ]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [ ]:
## Using solver0
from algorithm.koopmanlib.solver_0 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=100000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
## Results from solver0
evalues = solver.eigenvalues.T
# efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

# # Build the Koopman model (this will compute the Koopman matrix K and store it in self.K_reg)
# solver.build(data_train)

# After building the model, you can access the Koopman matrix K
Koopman_matrix_K = solver.K_reg.numpy()

resDMD_DL_outputs0 = {
    'resDMD_DL_outputs0': {
        # 'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'Psi_X': Psi_X.numpy(),
        'Psi_Y': Psi_Y.numpy(),
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

# print(resDMD_DL_outputs0['resDMD_DL_outputs']['evalues'].shape)
# print(resDMD_DL_outputs0['resDMD_DL_outputs']['efuns'].shape)

import os
# np.save("solver_0_outputs.npy", resDMD_DL_outputs0, allow_pickle=True)
save_path = 'D:\\Residual-Dynamic-Mode-Decomposition-main\\Examples_gallery_1\\ResDMD_datasets'  # Ensure this is a directory
nd = resDMD_DL_outputs0['resDMD_DL_outputs0']['evalues'].shape[0]
filename = f'solver0_outputs_{nd}basis.mat'
# scipy.io.savemat(save_path + filename, resDMD_DL_outputs0)
# Correctly join paths with os.path.join
full_path = os.path.join(save_path, filename)
# Save the .mat file
# scipy.io.savemat(full_path, {'resDMD_DL_outputs': resDMD_DL_outputs0})
# Assuming 'resDMD_DL_outputs' is the key you want to unpack
data_to_save = resDMD_DL_outputs0['resDMD_DL_outputs0']

# Save the .mat file with each key as its own variable
scipy.io.savemat(full_path, data_to_save)

In [ ]:
print(Koopman_matrix_K)

In [ ]:
## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()

## Check if basis is orthonormal or normailized
def is_orthonormal(A, tol=1e-6):
    # A should be orthonormal if A' * A = I
    A_H = np.conj(A).T  # Conjugate transpose for complex-valued A
    product = np.dot(A_H, A)
    I = np.eye(A.shape[1])  # Identity matrix
    return np.allclose(product, I, atol=tol)

# # Now, use this function to check orthonormality of Psi_X and Psi_Y
# print("Is Psi_X orthonormal?", is_orthonormal(Psi_X))
# print("Is Psi_Y orthonormal?", is_orthonormal(Psi_Y))

def are_columns_normalized(A, tol=1e-6):
    # Calculate the norm of each column
    norms = np.linalg.norm(A, axis=0)
    # Check if each norm is close to 1 within the specified tolerance
    return np.allclose(norms, 1, atol=tol)

# # Now, use this function to check the normalization of Psi_X and Psi_Y
# print("Are the columns of Psi_X normalized?", are_columns_normalized(Psi_X))
# print("Are the columns of Psi_Y normalized?", are_columns_normalized(Psi_Y))
# print(evalues)


In [ ]:
# Using solver3
from algorithm.koopmanlib.solver_3 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=100,
             batch_size=100000,
             lr=1e-5,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
## Results from solver3
evalues = solver.eigenvalues.T
# efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
# Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

# # Build the Koopman model (this will compute the Koopman matrix K and store it in self.K_reg)
# solver.build(data_train)

# After building the model, you can access the Koopman matrix K
Koopman_matrix_K = solver.K_reg.numpy()

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs3': {
        # 'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        # 'Psi_X': Psi_X.numpy(),
        # 'Psi_Y': Psi_Y.numpy(),
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

# print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'].shape)
# print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)

import os
# np.save("solver_3_outputs.npy", resDMD_DL_outputs0, allow_pickle=True)
save_path = 'D:\\Residual-Dynamic-Mode-Decomposition-main\\Examples_gallery_1\\ResDMD_datasets'  # Ensure this is a directory
nd = resDMD_DL_outputs3['resDMD_DL_outputs3']['evalues'].shape[0]
filename = f'solver3_outputs_{nd}basis.mat'
# scipy.io.savemat(save_path + filename, resDMD_DL_outputs3)
# Correctly join paths with os.path.join
full_path = os.path.join(save_path, filename)
# Save the .mat file
# scipy.io.savemat(full_path, {'resDMD_DL_outputs': resDMD_DL_outputs3})
# Assuming 'resDMD_DL_outputs' is the key you want to unpack
data_to_save = resDMD_DL_outputs3['resDMD_DL_outputs3']

# Save the .mat file with each key as its own variable
scipy.io.savemat(full_path, data_to_save)

In [ ]:
## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()

## Check if basis is orthonormal or normailized
def is_orthonormal(A, tol=1e-6):
    # A should be orthonormal if A' * A = I
    A_H = np.conj(A).T  # Conjugate transpose for complex-valued A
    product = np.dot(A_H, A)
    I = np.eye(A.shape[1])  # Identity matrix
    return np.allclose(product, I, atol=tol)

# # Now, use this function to check orthonormality of Psi_X and Psi_Y
# print("Is Psi_X orthonormal?", is_orthonormal(Psi_X))
# print("Is Psi_Y orthonormal?", is_orthonormal(Psi_Y))

def are_columns_normalized(A, tol=1e-6):
    # Calculate the norm of each column
    norms = np.linalg.norm(A, axis=0)
    # Check if each norm is close to 1 within the specified tolerance
    return np.allclose(norms, 1, atol=tol)

# # Now, use this function to check the normalization of Psi_X and Psi_Y
# print("Are the columns of Psi_X normalized?", are_columns_normalized(Psi_X))
# print("Are the columns of Psi_Y normalized?", are_columns_normalized(Psi_Y))
# print(evalues)


In [ ]:
# Using solver4
from algorithm.koopmanlib.solver_4 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=500,
             batch_size=100000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
# Results from solver4
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

# # Build the Koopman model (this will compute the Koopman matrix K and store it in self.K_reg)
# solver.build(data_train)

# After building the model, you can access the Koopman matrix K
Koopman_matrix_K = solver.K_reg.numpy()

resDMD_DL_outputs4 = {
    'resDMD_DL_outputs': {
        'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'Psi_X': Psi_X.numpy(),
        'Psi_Y': Psi_Y.numpy(),
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

# np.save("solver_4_outputs.npy", resDMD_DL_outputs4, allow_pickle=True)
save_path = 'D:\\resDMD\\data\\'  # Ensure this is a directory
nd = resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'].shape[0]
filename = f'solver4_outputs_{nd}basis.mat'
scipy.io.savemat(save_path + filename, resDMD_DL_outputs4)

In [ ]:
## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()

## Check if basis is orthonormal or normailized
def is_orthonormal(A, tol=1e-6):
    # A should be orthonormal if A' * A = I
    A_H = np.conj(A).T  # Conjugate transpose for complex-valued A
    product = np.dot(A_H, A)
    I = np.eye(A.shape[1])  # Identity matrix
    return np.allclose(product, I, atol=tol)

# # Now, use this function to check orthonormality of Psi_X and Psi_Y
# print("Is Psi_X orthonormal?", is_orthonormal(Psi_X))
# print("Is Psi_Y orthonormal?", is_orthonormal(Psi_Y))

def are_columns_normalized(A, tol=1e-6):
    # Calculate the norm of each column
    norms = np.linalg.norm(A, axis=0)
    # Check if each norm is close to 1 within the specified tolerance
    return np.allclose(norms, 1, atol=tol)

# # Now, use this function to check the normalization of Psi_X and Psi_Y
# print("Are the columns of Psi_X normalized?", are_columns_normalized(Psi_X))
# print("Are the columns of Psi_Y normalized?", are_columns_normalized(Psi_Y))
# print(evalues)


In [ ]:
# # Using solver1
# from algorithm.koopmanlib.solver_1 import KoopmanDLSolver # resDMD solver using regression
# solver = KoopmanDLSolver(dic=basis_function,
#                          target_dim=np.shape(data_train)[-1],
#                          reg=0.1)

# solver.build(data_train=data_train,
#              data_valid=data_valid,
#              epochs=1000,
#              batch_size=170000,
#              lr=1e-4,
#              log_interval=10,
#              lr_decay_factor=.8)

In [ ]:
# # Results from solver1
# evalues = solver.eigenvalues.T
# efuns = solver.eigenfunctions(X)
# kpm_modes = solver.compute_mode().T
# N_dict = np.shape(evalues)[0]
# Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

# # # Build the Koopman model (this will compute the Koopman matrix K and store it in self.K_reg)
# # solver.build(data_train)

# # After building the model, you can access the Koopman matrix K
# Koopman_matrix_K = solver.K_reg.numpy()

# resDMD_DL_outputs1 = {
#     'resDMD_DL_outputs': {
#         'efuns': efuns,
#         'evalues': evalues,
#         'kpm_modes': kpm_modes,
#         'N_dict': N_dict,
#         'Psi_X': Psi_X.numpy(),
#         'Psi_Y': Psi_Y.numpy(),
#         'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
#     }
# }

# # np.save("solver_1_outputs.npy", resDMD_DL_outputs1, allow_pickle=True)
# save_path = 'D:\\resDMD\\data\\'  # Ensure this is a directory
# nd = resDMD_DL_outputs1['resDMD_DL_outputs']['evalues'].shape[0]
# filename = f'solver1_outputs_{nd}basis.mat'
# scipy.io.savemat(save_path + filename, resDMD_DL_outputs1)